<a href="https://colab.research.google.com/github/DEEPAK-RAMGIRI/ORGANIC_RECCOMENDATOR/blob/main/MP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Extraction

In [1]:
import pandas as pd

In [2]:
sheet_id = '1QPf1JVbH3YxXYsh4bnitfdMAyro-SylYlWSHBiY2wQU'
fileUrl = f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv'

In [3]:
try:
  data = pd.read_csv(fileUrl)
  print("OK😊👍")
except Exception as e:
  print(f"Error loading data: {e}")

OK😊👍


In [4]:
data.head(10)

,chemical_name,chemical_type,crop,problem_or_pest,organic_alternative,dosage,application_time,safety_note
0,DAP,Fertilizer,Maize,Phosphorus deficiency,Rock phosphate,200 kg/acre,Basal application,Safe when properly composted / diluted
1,MOP,Fertilizer,Chilli,Potassium deficiency,Wood ash,150 kg/acre,Soil application,Safe when properly composted / diluted
2,Glyphosate,Herbicide,Cotton,Weeds,Mulching,Organic mulch,Throughout season,Safe when properly composted / diluted
3,Mancozeb,Fungicide,Maize,Leaf blight,Pseudomonas fluorescens,5 g/L,Foliar spray,Safe when properly composted / diluted
4,DAP,Fertilizer,Groundnut,Phosphorus deficiency,Rock phosphate,200 kg/acre,Basal application,Safe when properly composted / diluted
5,Urea,Fertilizer,Chilli,Nitrogen deficiency,Green manure,Incorporation,30 days before planting,Safe when properly composted / diluted
6,MOP,Fertilizer,Maize,Potassium deficiency,Wood ash,150 kg/acre,Soil application,Safe when properly composted / diluted
7,Chlorpyrifos,Pesticide,Groundnut,Aphids,Neem oil,3 ml/L,Early morning,Safe when properly composted / diluted
8,Chlorpyrifos,Pesticide,Chilli,Aphids,Neem oil,3 ml/L,Early morning,Safe when properly composted / diluted
9,Imidacloprid,Pesticide,Rice,Whiteflies,Neem oil,3 ml/L,Early morning,Safe when properly composted / diluted


In [5]:
data.tail(10)

,chemical_name,chemical_type,crop,problem_or_pest,organic_alternative,dosage,application_time,safety_note
1490,Mancozeb,Fungicide,Tomato,Leaf blight,Pseudomonas fluorescens,5 g/L,Foliar spray,Safe when properly composted / diluted
1491,Imidacloprid,Pesticide,Wheat,Whiteflies,Neem oil,3 ml/L,Early morning,Safe when properly composted / diluted
1492,Mancozeb,Fungicide,Rice,Leaf blight,Pseudomonas fluorescens,5 g/L,Foliar spray,Safe when properly composted / diluted
1493,DAP,Fertilizer,Maize,Phosphorus deficiency,Bone meal,250 kg/acre,Basal application,Safe when properly composted / diluted
1494,Mancozeb,Fungicide,Rice,Leaf blight,Pseudomonas fluorescens,5 g/L,Foliar spray,Safe when properly composted / diluted
1495,Imidacloprid,Pesticide,Maize,Whiteflies,Neem oil,3 ml/L,Early morning,Safe when properly composted / diluted
1496,DAP,Fertilizer,Rice,Phosphorus deficiency,Bone meal,250 kg/acre,Basal application,Safe when properly composted / diluted
1497,Carbendazim,Fungicide,Chilli,Fungal disease,Trichoderma,10 g/kg seed,Before sowing,Safe when properly composted / diluted
1498,Mancozeb,Fungicide,Groundnut,Leaf blight,Pseudomonas fluorescens,5 g/L,Foliar spray,Safe when properly composted / diluted
1499,Chlorpyrifos,Pesticide,Rice,Aphids,NSKE,5% solution,Evening,Safe when properly composted / diluted


In [6]:
data.shape

(1500, 8)

In [7]:
data.isnull()
data.isnull().sum()

,0
chemical_name,0
chemical_type,0
crop,0
problem_or_pest,0
organic_alternative,0
dosage,0
application_time,0
safety_note,0


## Data preprocessing


In [8]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
data.head(0)

,chemical_name,chemical_type,crop,problem_or_pest,organic_alternative,dosage,application_time,safety_note


In [10]:
# we are using chemical_name, crop, and problem_or_pest
data['chemical_name'] = data['chemical_name'].str.lower().replace(r'^\w\s',' ').replace(r'\n',' ',regex = True)
data['crop'] = data['crop'].str.lower().replace(r'^\w\s',' ').replace(r'\n',' ',regex = True)
data['problem_or_pest'] = data['problem_or_pest'].str.lower().replace(r'^\w\s',' ').replace(r'\n',' ',regex = True)

In [11]:
data['combined_features'] = (data['chemical_name'] + " " + data['crop']+ " " + data['problem_or_pest'])

In [12]:
data['combined_features'].head(2)

,combined_features
0,dap maize phosphorus deficiency
1,mop chilli potassium deficiency


In [16]:
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [17]:
lemmatizer = WordNetLemmatizer()
def tokenize_words(text):
  tokens = word_tokenize(text)
  # stemmed_tokens = [stemming.stem(i) for i in token_words]
  clean_tokens = [lemmatizer.lemmatize(w) for w in tokens if w.isalpha()]
  return ' '.join(clean_tokens)

In [18]:
data['combined_features'].apply(lambda i: tokenize_words(i))

,combined_features
0,dap maize phosphorus deficiency
1,mop chilli potassium deficiency
2,glyphosate cotton weed
3,mancozeb maize leaf blight
4,dap groundnut phosphorus deficiency
...,...
1495,imidacloprid maize whitefly
1496,dap rice phosphorus deficiency
1497,carbendazim chilli fungal disease
1498,mancozeb groundnut leaf blight


## Reccomending

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
# tf = tfidf(analyzer='word',stop_words='english')
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
vectors = tfidf.fit_transform(data['combined_features'])
similar = cosine_similarity(vectors)

In [21]:
import re

def reccomend(chemical, crop, acres):
    query_text = f"{chemical} {crop}".lower()
    query_vec = tfidf.transform([query_text])

    similarity = cosine_similarity(query_vec, vectors)
    best_match_idx = similarity.argmax()

    res = data.iloc[best_match_idx]

    try:

        base_num = float(re.search(r"(\d+\.?\d*)", res['dosage']).group(1))

        if 'acre' in res['dosage'].lower():
            total_needed = base_num * acres
            unit = res['dosage'].split('/')[0].replace(str(int(base_num)) if base_num.is_integer() else str(base_num), "").strip()
            calculated_dosage = f"{total_needed} {unit} total for {acres} acres"
        else:
            calculated_dosage = f"{res['dosage']} (Mix according to water volume)"
    except:
        calculated_dosage = res['dosage']

    return {
        "Input": f"{chemical} on {crop}",
        "Organic Alternative": res['organic_alternative'],
        "Calculated Quantity": calculated_dosage,
        "Application Time": res['application_time'],
        "Safety Note": res['safety_note'],
        "Confidence Score": round(similarity[0][best_match_idx], 4)
    }

# Testing the corrected function
print(reccomend("Urea", "Cotton", 5.0))


{'Input': 'Urea on Cotton', 'Organic Alternative': 'Green manure', 'Calculated Quantity': 'Incorporation', 'Application Time': '30 days before planting', 'Safety Note': 'Safe when properly composted / diluted', 'Confidence Score': np.float64(0.6864)}


In [23]:
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(tfidf, f)
with open('vectors.pkl', 'wb') as f:
    pickle.dump(vectors, f)
with open('data.pkl', 'wb') as f:
    pickle.dump(data, f)